In [165]:
import pandas as pd

In [166]:
# read 1998 - april 2023 AQS data
df = pd.read_csv('source/chicago-cbsa-pm25-aqi-1998-2023.csv')

/tmp/ipykernel_147924/1558418500.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('source/chicago-cbsa-pm25-aqi-1998-2023.csv')


In [167]:
# convert date
df['date_local'] = pd.to_datetime(df['date_local'])

In [175]:
# create month year date col
df['month_year'] = df['date_local'].dt.strftime('%m/%Y')

In [169]:
# inspect 
df[df['local_site_name'] == 'SPRINGFIELD PUMP STATION'][['date_local', 'local_site_name', 'aqi','pollutant_standard']].sort_values('date_local', ascending=False).tail(5)

,date_local,local_site_name,aqi,pollutant_standard
40072,2000-01-13,SPRINGFIELD PUMP STATION,48.0,PM25 24-hour 2012
40071,2000-01-13,SPRINGFIELD PUMP STATION,48.0,PM25 Annual 2006
40074,2000-01-13,SPRINGFIELD PUMP STATION,48.0,PM25 24-hour 1997
40075,2000-01-13,SPRINGFIELD PUMP STATION,48.0,PM25 Annual 1997
40070,2000-01-13,SPRINGFIELD PUMP STATION,48.0,PM25 24-hour 2006


In [317]:
cbsa_timeseries[['date_local'] > '2023-01-01'].groupby('local_site_name').size()

TypeError: '>' not supported between instances of 'list' and 'str'

In [189]:
# get local site names in Chicago that track PM 2.5
# https://www.epa.gov/sites/default/files/2019-07/documents/2019_illinois_network_plan.pdf#page=19
chicago_pm25_sites = ['WASHINGTON HS', 'MAYFAIR PUMP STATION', 'SPRINGFIELD PUMP STATION', 'COM ED MAINTENANCE BLDG']

In [190]:
# find earliest day for each site
print('WASHINGTON HS:', df[df['local_site_name'] == 'WASHINGTON HS'][['date_local']].sort_values('date_local', ascending=False).tail(1))
print('MAYFAIR PUMP STATION:', df[df['local_site_name'] == 'MAYFAIR PUMP STATION'][['date_local']].sort_values('date_local', ascending=False).tail(1))
print('SPRINGFIELD PUMP STATION:', df[df['local_site_name'] == 'SPRINGFIELD PUMP STATION'][['date_local']].sort_values('date_local', ascending=False).tail(1))
print('COM ED MAINTENANCE BLDG:', df[df['local_site_name'] == 'COM ED MAINTENANCE BLDG'][['date_local']].sort_values('date_local', ascending=False).tail(1))

WASHINGTON HS:       date_local
12450 1999-01-06
MAYFAIR PUMP STATION:     date_local
324 1998-01-05
SPRINGFIELD PUMP STATION:       date_local
40070 2000-01-13
COM ED MAINTENANCE BLDG:       date_local
16290 2000-01-01


In [191]:
# find most recent day for each site
print('WASHINGTON HS:', df[df['local_site_name'] == 'WASHINGTON HS'][['date_local']].sort_values('date_local', ascending=False).head(1))
print('MAYFAIR PUMP STATION:', df[df['local_site_name'] == 'MAYFAIR PUMP STATION'][['date_local']].sort_values('date_local', ascending=False).head(1))
print('SPRINGFIELD PUMP STATION:', df[df['local_site_name'] == 'SPRINGFIELD PUMP STATION'][['date_local']].sort_values('date_local', ascending=False).head(1))
print('COM ED MAINTENANCE BLDG:', df[df['local_site_name'] == 'COM ED MAINTENANCE BLDG'][['date_local']].sort_values('date_local', ascending=False).head(1))

WASHINGTON HS:        date_local
650579 2023-04-30
MAYFAIR PUMP STATION:        date_local
653774 2023-04-30
SPRINGFIELD PUMP STATION:        date_local
651539 2023-04-30
COM ED MAINTENANCE BLDG:        date_local
649699 2023-04-30


In [173]:
# figure out which pollutant standard to use: PM25 24 hour 2012, according to EPA

In [174]:
# how many rows does each pollutant standard have?
print('PM25 24-hour 2012:', len(df[df['pollutant_standard'] == 'PM25 24-hour 2012']))
print('PM25 24-hour 2006:', len(df[df['pollutant_standard'] == 'PM25 24-hour 2006']))
print('PM25 Annual 2012:', len(df[df['pollutant_standard'] == 'PM25 Annual 2012']))
print('PM25 Annual 1997:', len(df[df['pollutant_standard'] == 'PM25 Annual 1997']))

PM25 24-hour 2012: 107928
PM25 24-hour 2006: 107928
PM25 Annual 2012: 93739
PM25 Annual 1997: 107950


In [192]:
# make sure the earliest date for the PM25 24-hour 2012 pollutant standard is the earliest date for each site
print('WASHINGTON HS:', df[(df['local_site_name'] == 'WASHINGTON HS') & (df['pollutant_standard'] == 'PM25 24-hour 2012')][['date_local']].sort_values('date_local', ascending=False).tail(1))
print('MAYFAIR PUMP STATION:', df[(df['local_site_name'] == 'MAYFAIR PUMP STATION') & (df['pollutant_standard'] == 'PM25 24-hour 2012')][['date_local']].sort_values('date_local', ascending=False).tail(1))
print('SPRINGFIELD PUMP STATION:', df[(df['local_site_name'] == 'SPRINGFIELD PUMP STATION') & (df['pollutant_standard'] == 'PM25 24-hour 2012')][['date_local']].sort_values('date_local', ascending=False).tail(1))
print('COM ED MAINTENANCE BLDG:', df[(df['local_site_name'] == 'COM ED MAINTENANCE BLDG') & (df['pollutant_standard'] == 'PM25 24-hour 2012')][['date_local']].sort_values('date_local', ascending=False).tail(1))

WASHINGTON HS:       date_local
12452 1999-01-06
MAYFAIR PUMP STATION:     date_local
326 1998-01-05
SPRINGFIELD PUMP STATION:       date_local
40072 2000-01-13
COM ED MAINTENANCE BLDG:       date_local
16292 2000-01-01


In [207]:
# assign chicago vs suburbs geography based on local site name
def get_geo(row):
    if row['local_site_name'] == 'WASHINGTON HS' or row['local_site_name'] == 'MAYFAIR PUMP STATION' or row['local_site_name'] == 'SPRINGFIELD PUMP STATION' or row['local_site_name'] == 'COM ED MAINTENANCE BLDG':
        return 'Chicago'
    else:
        return 'Suburbs'

df['geography'] = df.apply(get_geo, axis=1)

In [194]:
len(df[df['geography'] == 'Chicago'])

64188

In [266]:
# filter dataframe for just 2012 pollutant standard and geography
chi_df = df[(df['pollutant_standard'] == 'PM25 24-hour 2012') & (df['geography'] == 'Chicago')]
burbs_df = df[(df['pollutant_standard'] == 'PM25 24-hour 2012') & (df['geography'] == 'Suburbs')]
cbsa_df = df[(df['pollutant_standard'] == 'PM25 24-hour 2012')]

In [267]:
# filter for just necessary cols for flourish
chi_df_viz = chi_df[['date_local', 'aqi', 'local_site_name']].copy()
burbs_df_viz = burbs_df[['date_local', 'aqi', 'local_site_name']].copy()
cbsa_df_viz = cbsa_df[['date_local', 'aqi', 'local_site_name']].copy()

In [248]:
# load AirNow data - update this daily
airnow_df = pd.read_csv('source/2023_PM2.5_CBSA_6.28.23.csv')

In [249]:
# rename cols
airnow_df = airnow_df.rename(columns={'Date': 'date_local', 'DAILY_AQI_VALUE': 'aqi', 'Site Name': 'local_site_name'})

In [250]:
# convert date
airnow_df['date_local'] = pd.to_datetime(airnow_df['date_local'])

# create month year date col
airnow_df['month_year'] = airnow_df['date_local'].dt.strftime('%m/%Y')

In [251]:
# assign geography
airnow_df['geography'] = airnow_df.apply(get_geo, axis=1)

In [268]:
# create suburbs and chicago
chi_airnow_df = airnow_df[airnow_df['geography'] == 'Chicago']
burbs_airnow_df = airnow_df[airnow_df['geography'] == 'Suburbs']
cbsa_airnow_df = airnow_df

In [269]:
# cut down to the few columns
chi_airnow_df_appended = chi_airnow_df[['date_local', 'aqi', 'local_site_name']].copy()
burbs_airnow_df_appended = burbs_airnow_df[['date_local', 'aqi', 'local_site_name']].copy()
cbsa_airnow_df_appended = cbsa_airnow_df[['date_local', 'aqi', 'local_site_name']].copy()

In [270]:
# only include dates after 4/30
chi_airnow_df_appended = chi_airnow_df_appended[chi_airnow_df_appended['date_local'] > '04-30-2023']
burbs_airnow_df_appended = burbs_airnow_df_appended[burbs_airnow_df_appended['date_local'] > '04-30-2023']
cbsa_airnow_df_appended = cbsa_airnow_df_appended[cbsa_airnow_df_appended['date_local'] > '04-30-2023']

In [271]:
# append airnow with AQS data
chi_timeseries = pd.concat([chi_df_viz, chi_airnow_df_appended])
burbs_timeseries = pd.concat([burbs_df_viz, burbs_airnow_df_appended])
cbsa_timeseries = pd.concat([cbsa_df_viz, cbsa_airnow_df_appended])

In [272]:
cbsa_timeseries.sort_values('date_local').tail(10)

,date_local,aqi,local_site_name
8,2023-06-28,203.0,CITY HALL
6,2023-06-28,194.0,NORTHBROOK WATER PLANT
7,2023-06-28,173.0,LIBERTY SCHOOL
9,2023-06-28,184.0,CARY GROVE HS
10,2023-06-28,221.0,PERSHING SCHOOL
11,2023-06-28,209.0,COM ED TRAINING CENTER
12,2023-06-28,169.0,Gary-IITRI/ 1219.5 meters east of Tennessee St...
14,2023-06-28,180.0,Hammond- 167th St
5,2023-06-28,194.0,REGIONAL OFFICE BUILDING
15,2023-06-28,166.0,Ogden Dunes- Water Treatment Plant


In [273]:
# export
chi_timeseries.to_csv('processed/chi_timeseries_6.28.23.csv')
burbs_timeseries.to_csv('processed/burbs_timeseries_6.28.23.csv')
cbsa_timeseries.to_csv('processed/cbsa_timeseries_6.28.23.csv')

In [ ]:
# transpose just chi data for the flourish line chart

In [259]:
# create individual site dfs
washington = chi_timeseries[chi_timeseries['local_site_name'] == 'WASHINGTON HS'].copy()
mayfair = chi_timeseries[chi_timeseries['local_site_name'] == 'MAYFAIR PUMP STATION'].copy()
springfield = chi_timeseries[chi_timeseries['local_site_name'] == 'SPRINGFIELD PUMP STATION'].copy()
comed = chi_timeseries[chi_timeseries['local_site_name'] == 'COM ED MAINTENANCE BLDG'].copy()

In [260]:
# rename aqi cols to prep for merge
washington = washington.rename(columns={'aqi': 'WASHINGTON HS'})
mayfair = mayfair.rename(columns={'aqi': 'MAYFAIR PUMP STATION'})
springfield = springfield.rename(columns={'aqi': 'SPRINGFIELD PUMP STATION'})
comed = comed.rename(columns={'aqi': 'COM ED MAINTENANCE BLDG'})

In [261]:
viz_df_transposed = pd.merge( washington[['date_local', 'WASHINGTON HS']], mayfair[['date_local','MAYFAIR PUMP STATION']], on='date_local', how='outer')
viz_df_transposed = pd.merge(viz_df_transposed, springfield[['date_local','SPRINGFIELD PUMP STATION']], on='date_local', how='outer')
viz_df_transposed = pd.merge(viz_df_transposed, comed[['date_local','COM ED MAINTENANCE BLDG']], on='date_local', how='outer')

In [262]:
viz_df_transposed

,date_local,WASHINGTON HS,MAYFAIR PUMP STATION,SPRINGFIELD PUMP STATION,COM ED MAINTENANCE BLDG
0,1999-01-06,84.0,81.0,NaN,NaN
1,1999-01-24,75.0,69.0,NaN,NaN
2,1999-01-30,93.0,86.0,NaN,NaN
3,1999-02-05,58.0,73.0,NaN,NaN
4,1999-02-14,54.0,NaN,NaN,NaN
...,...,...,...,...,...
5747,2021-03-06,NaN,NaN,NaN,47.0
5748,2021-05-05,NaN,NaN,NaN,12.0
5749,2022-06-18,NaN,NaN,NaN,15.0
5750,2022-10-19,NaN,NaN,NaN,15.0


In [263]:
# export transposed data for flourish
viz_df_transposed.to_csv('processed/chi_timeseries_transposed_6.28.23.csv')

In [287]:
# create an averages for cbsa timeseries for flourish scatterplot
cbsa_timeseries_avg_by_date = cbsa_timeseries.groupby('date_local')['aqi'].mean().reset_index(name = 'avg_aqi')

In [290]:
cbsa_timeseries_avg_by_date = cbsa_timeseries_avg_by_date.sort_values('date_local')

In [291]:
cbsa_timeseries_avg_by_date

,date_local,avg_aqi
0,1998-01-05,38.500000
1,1998-01-11,46.500000
2,1998-01-17,94.500000
3,1998-01-23,92.000000
4,1998-01-29,71.000000
...,...,...
9063,2023-06-24,59.529412
9064,2023-06-25,39.058824
9065,2023-06-26,90.411765
9066,2023-06-27,185.235294


In [295]:
cbsa_timeseries_avg_by_date.sort_values('avg_aqi').head(5)

,date_local,avg_aqi
491,1999-08-30,1.000000
5186,2012-11-03,5.750000
5388,2013-05-24,6.333333
279,1999-01-21,7.000000
2517,2005-04-24,7.000000


In [293]:
# export cbsa daily averages across sensors
cbsa_timeseries_avg_by_date.to_csv('processed/cbsa_timeseries_daily_avg_across_sensors_6.28.23.csv')

In [296]:
# only one day on April 20, 2016 in Gary exceeded the pollution levels seen this week
cbsa_timeseries.sort_values('aqi').tail(20)

,date_local,aqi,local_site_name
21,2023-06-27,184.0,IEPA TRAILER
28,2023-06-27,184.0,COM ED TRAINING CENTER
333656,2013-11-15,186.0,Ogden Dunes- Water Treatment Plant
30,2023-06-27,186.0,East Chicago-Marina
31,2023-06-27,188.0,Hammond- 167th St
643277,2022-11-08,188.0,Ogden Dunes- Water Treatment Plant
33,2023-06-27,190.0,CHIWAUKEE PRAIRIE STATELINE
23,2023-06-27,191.0,NORTHBROOK WATER PLANT
20,2023-06-27,193.0,Kingery Near Road #1
6,2023-06-28,194.0,NORTHBROOK WATER PLANT


In [277]:
## could count the number of days where the max is over very unhealthy for cbsa vs just chicago
cbsa_max_by_date = cbsa_timeseries.groupby('date_local')['aqi'].max().reset_index(name = 'max')

In [280]:
cbsa_max_by_date

,date_local,max
0,1998-01-05,48.0
1,1998-01-11,50.0
2,1998-01-17,107.0
3,1998-01-23,126.0
4,1998-01-29,74.0
...,...,...
9063,2023-06-24,76.0
9064,2023-06-25,47.0
9065,2023-06-26,104.0
9066,2023-06-27,209.0


In [314]:
# on only 3 days since 1998 has the aqi exceeded very unhealthy levels in the entire chicago metro area
cbsa_max_by_date[cbsa_max_by_date['max'] > 200]

,date_local,max
6442,2016-04-20,217.0
9066,2023-06-27,209.0
9067,2023-06-28,221.0


In [328]:
len(cbsa_max_by_date[cbsa_max_by_date['max'] > 150])

39

In [ ]:
len(cbsa_max_by_date[cbsa_max_by_date['max'] < 150])

In [327]:
cbsa_max_by_date.sort_values('max', ascending=False).head(10)

,date_local,max
9067,2023-06-28,221.0
6442,2016-04-20,217.0
9066,2023-06-27,209.0
8835,2022-11-08,188.0
5560,2013-11-15,186.0
6156,2015-07-05,183.0
8250,2021-04-02,169.0
7028,2017-11-27,167.0
6274,2015-11-04,164.0
4966,2012-03-27,162.0


In [298]:
# what sensor wasa that 2016 day?
cbsa_timeseries[cbsa_timeseries['date_local'] == '2016-04-20']

,date_local,aqi,local_site_name
386071,2016-04-20,41.0,MAYFAIR PUMP STATION
397124,2016-04-20,217.0,Gary-IITRI/ 1219.5 meters east of Tennessee St...
397130,2016-04-20,217.0,Gary-IITRI/ 1219.5 meters east of Tennessee St...
401081,2016-04-20,33.0,Ogden Dunes- Water Treatment Plant


In [282]:
# on 39 days since 1998 has the aqi exceeded very unhealthy levels in the entire chicago metro area
len(cbsa_max_by_date[cbsa_max_by_date['max'] > 150])

39

In [283]:
# 24 of those days occurred before 2010
cbsa_max_by_date[cbsa_max_by_date['max'] > 150]

,date_local,max
52,1998-05-15,152.0
403,1999-05-30,157.0
426,1999-06-23,151.0
464,1999-08-02,153.0
634,2000-01-20,153.0
639,2000-01-25,154.0
894,2000-10-06,154.0
911,2000-10-23,155.0
981,2001-01-01,151.0
985,2001-01-05,158.0


In [301]:
chi_max_by_date = chi_timeseries.groupby('date_local')['aqi'].max().reset_index(name = 'max')

In [307]:
# In the City of Chicago, the highest AQI ever recorded was on June 27 at 171
chi_max_by_date.sort_values('max', ascending=False).head()

,date_local,max
5471,2023-06-27,171.0
5472,2023-06-28,170.0
4251,2015-07-05,170.0
611,2000-10-23,155.0
2076,2005-02-03,155.0


In [309]:
# sensor highs on 27 and 28 of june
cbsa_timeseries[(cbsa_timeseries['date_local'] == '2023-06-27')].sort_values('aqi', ascending=False)

,date_local,aqi,local_site_name
25,2023-06-27,209.0,CITY HALL
26,2023-06-27,205.0,CARY GROVE HS
27,2023-06-27,199.0,PERSHING SCHOOL
22,2023-06-27,199.0,REGIONAL OFFICE BUILDING
20,2023-06-27,193.0,Kingery Near Road #1
23,2023-06-27,191.0,NORTHBROOK WATER PLANT
33,2023-06-27,190.0,CHIWAUKEE PRAIRIE STATELINE
31,2023-06-27,188.0,Hammond- 167th St
30,2023-06-27,186.0,East Chicago-Marina
21,2023-06-27,184.0,IEPA TRAILER


In [310]:
cbsa_timeseries[(cbsa_timeseries['date_local'] == '2023-06-28')].sort_values('aqi', ascending=False)

,date_local,aqi,local_site_name
10,2023-06-28,221.0,PERSHING SCHOOL
11,2023-06-28,209.0,COM ED TRAINING CENTER
8,2023-06-28,203.0,CITY HALL
5,2023-06-28,194.0,REGIONAL OFFICE BUILDING
6,2023-06-28,194.0,NORTHBROOK WATER PLANT
9,2023-06-28,184.0,CARY GROVE HS
3,2023-06-28,184.0,Kingery Near Road #1
14,2023-06-28,180.0,Hammond- 167th St
13,2023-06-28,179.0,East Chicago-Marina
16,2023-06-28,179.0,CHIWAUKEE PRAIRIE STATELINE


In [322]:
cbsa_timeseries[(cbsa_timeseries['date_local'] > '2023-01-01')].groupby('local_site_name').size().reset_index()

,local_site_name,0
0,CARY GROVE HS,178
1,CHIWAUKEE PRAIRIE STATELINE,147
2,CITY HALL,178
3,COM ED MAINTENANCE BLDG,69
4,COM ED TRAINING CENTER,178
5,East Chicago- Franklin School/ Washington Scho...,40
6,East Chicago-Marina,214
7,GRAVES ES,59
8,Gary- Burr St./ pumping station near truck stop,80
9,Gary- Madison St./ Gary Water/ IN American Wat...,53


In [324]:
len(cbsa_timeseries.groupby('local_site_name').size().reset_index())

35

In [325]:
cbsa_timeseries.groupby('local_site_name').size().reset_index()

,local_site_name,0
0,CAMP LOGAN TRAILER,818
1,CARY GROVE HS,2378
2,CHIWAUKEE PRAIRIE STATELINE,4846
3,CITY HALL,3227
4,COM ED MAINTENANCE BLDG,1824
5,COM ED TRAINING CENTER,3097
6,EISENHOWER HS,1321
7,East Chicago High School,18
8,East Chicago- Franklin School/ Washington Scho...,3555
9,East Chicago-Marina,1714


In [330]:
g = cbsa_timeseries.groupby('date_local')['aqi'].max().reset_index(name = 'max')
len(g[g['max'] < 100])

8733

In [331]:
len(g)

9068

In [332]:
8733/9068

0.9630569033965594

In [336]:
g[g['max'] < 100].sort_values('max').tail()

,date_local,max
2248,2004-07-14,99.0
856,2000-08-29,99.0
1737,2003-01-30,99.0
5066,2012-07-05,99.0
853,2000-08-26,99.0


In [338]:
len(g[g['max'] < 50])

3431

In [344]:
g[g['max'] < 50].sort_values('date_local').tail(20)

,date_local,max
9008,2023-04-30,22.0
9010,2023-05-02,16.0
9011,2023-05-03,35.0
9012,2023-05-04,44.0
9013,2023-05-05,32.0
9014,2023-05-06,40.0
9017,2023-05-09,49.0
9022,2023-05-14,49.0
9023,2023-05-15,39.0
9025,2023-05-17,30.0


In [378]:
cbsa_timeseries[cbsa_timeseries['date_local'] == '2023-06-26'].sort_values('aqi', ascending=False)

,date_local,aqi,local_site_name,date_formatted
42,2023-06-26,104.0,CITY HALL,06/2023
50,2023-06-26,102.0,CHIWAUKEE PRAIRIE STATELINE,06/2023
37,2023-06-26,102.0,Kingery Near Road #1,06/2023
40,2023-06-26,101.0,NORTHBROOK WATER PLANT,06/2023
48,2023-06-26,99.0,Hammond- 167th St,06/2023
39,2023-06-26,98.0,REGIONAL OFFICE BUILDING,06/2023
44,2023-06-26,98.0,PERSHING SCHOOL,06/2023
47,2023-06-26,96.0,East Chicago-Marina,06/2023
49,2023-06-26,90.0,Ogden Dunes- Water Treatment Plant,06/2023
46,2023-06-26,89.0,Gary-IITRI/ 1219.5 meters east of Tennessee St...,06/2023


In [339]:
3431/9068

0.3783634759594177

In [361]:
len(g.loc[(g['max']>=50) & (g['max']< 100)].sort_values('max', ascending=True))

5302

In [362]:
5302/9068

0.5846934274371416

In [363]:
0.58 + 0.37

0.95

In [364]:
chi_timeseries[['date_local'] ].sort_values('date_local')

,date_local,aqi,local_site_name
326,1998-01-05,48.0,MAYFAIR PUMP STATION
332,1998-01-11,43.0,MAYFAIR PUMP STATION
338,1998-01-17,107.0,MAYFAIR PUMP STATION
344,1998-01-23,126.0,MAYFAIR PUMP STATION
350,1998-01-29,74.0,MAYFAIR PUMP STATION
...,...,...,...
35,2023-06-26,81.0,SPRINGFIELD PUMP STATION
19,2023-06-27,167.0,COM ED MAINTENANCE BLDG
18,2023-06-27,171.0,SPRINGFIELD PUMP STATION
1,2023-06-28,170.0,SPRINGFIELD PUMP STATION


In [365]:
len(cbsa_timeseries)

108915

In [366]:
cbsa_timeseries.head()

,date_local,aqi,local_site_name
2,1998-01-23,72.0,NORTHBROOK WATER PLANT
8,1998-01-29,67.0,NORTHBROOK WATER PLANT
14,1998-02-10,98.0,NORTHBROOK WATER PLANT
20,1998-02-16,66.0,NORTHBROOK WATER PLANT
26,1998-02-22,79.0,NORTHBROOK WATER PLANT


In [367]:
cbsa_timeseries['date_formatted'] = cbsa_timeseries['date_local'].dt.strftime('%m/%Y')

In [376]:
cbsa_timeseries = cbsa_timeseries.sort_values('date_local', ascending=True)

In [377]:
# export with modified date
cbsa_timeseries.to_csv('processed/cbsa_timeseries_6.28.23.csv')